In [1]:
%load_ext autoreload
%autoreload 2

%load_ext memory_profiler

In [2]:
# Check output - should be ARM!
# If it is i386 (and you have M series machine) then you are using a non-native Python. Switch your Python to a native Python. A good way to do this is with Conda.


!python -c "import platform; print(platform.processor())"


arm


In [3]:
# Check current memory usage
from utils import MPS_MemoryTracker

with MPS_MemoryTracker():
    pass

Python (GC) and MPS cache emptied
######## Memory consumption:
        MPS tensors    MPS Total    Process Memory
------  -------------  -----------  ----------------
Before  0              0            243
After   0              0            243
Diff    +0 MB          +0 MB        +0 MB


# Download weights and convert them into MLX format

In [4]:
%%memit
 
from phi2_mlx import convert
# download weights and convert them into MLX format

with MPS_MemoryTracker(clean_cache_before=True, clean_cache_after=True):
    convert()

# Unfortunately, this function leaks memory, which could be seen in the ouput of memory tracker 

# objgraph.show_growth()

Python (GC) and MPS cache emptied


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Python (GC) and MPS cache emptied
######## Memory consumption:
        MPS tensors    MPS Total    Process Memory
------  -------------  -----------  ----------------
Before  0              0            266
After   0              0            2896
Diff    +0 MB          +0 MB        +2630 MB
peak memory: 9298.23 MiB, increment: 9054.27 MiB


# Load the model

In [5]:
from phi2_mlx import load_model, generate, get_tokenizer


In [6]:
%%memit

if 'model' in locals():
    print('Delete model obj')
    del model

with MPS_MemoryTracker():
    model = load_model()

Python (GC) and MPS cache emptied
######## Memory consumption:
        MPS tensors    MPS Total    Process Memory
------  -------------  -----------  ----------------
Before  0              0            2898
After   0              5560         6264
Diff    +0 MB          +5560 MB     +3366 MB
peak memory: 6457.02 MiB, increment: 3559.16 MiB


# Tokenize input

In [7]:
from transformers import AutoTokenizer


input_txt = 'What are the prime factors of 10?'

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
inputs = tokenizer(input_txt, return_tensors="np", return_attention_mask=False)
inputs


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': array([[2061,  389,  262, 6994, 5087,  286,  838,   30]])}

# Model inference

In [8]:
import mlx.core as mx

def generate_tokens(inputs, temp, max_tokens,
                    eos_token_id=tokenizer.eos_token_id
                    ):
    inputs = mx.array(inputs["input_ids"])

    tokens = []
    for token, ind in zip(generate(inputs, model, temp), range(max_tokens)):
        if token.item() == eos_token_id:
            print(f'---DEBUG--- EOS generated at {ind} position')
            break
        tokens.append(token)
    return tokens

In [9]:
# %%memit  # to use memit here, set TOKENIZERS_PARALLELISM=false


# Run `make track` during cell execution in mesure CPU/GPU usage

with MPS_MemoryTracker(clean_cache_before=True, clean_cache_after=True):
    tokens = generate_tokens(inputs, temp=0.2, max_tokens=200, eos_token_id=-1)

len(tokens)

Python (GC) and MPS cache emptied
Python (GC) and MPS cache emptied
######## Memory consumption:
        MPS tensors    MPS Total    Process Memory
------  -------------  -----------  ----------------
Before  0              5560         6264
After   0              10052        5295
Diff    +0 MB          +4492 MB     -969 MB


200

# Time measure

In [10]:
%%timeit

generate_tokens(inputs, temp=0.2, max_tokens=200, eos_token_id=-1)

7.13 s ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Full text-to-text example

In [11]:


def generate_txt(input_txt: str, tokenizer, temp=0.2, max_tokens=200):
    inputs = tokenizer(
        input_txt,
        return_tensors="np",
        return_attention_mask=False,
    )


    print("[INFO] Generating with Phi-2 on MLX...", flush=True)
    print(input_txt, end="", flush=True)

    tokens = generate_tokens(inputs, temp, max_tokens, eos_token_id=tokenizer.eos_token_id)

    mx.eval(tokens)
    s = tokenizer.decode([t.item() for t in tokens])
    return s    

input_txt = """
## INPUT
Write a short poem about deep learning
## Output
"""
text = generate_txt(input_txt, tokenizer, max_tokens=1000,)
print(text)


[INFO] Generating with Phi-2 on MLX...



## INPUT
Write a short poem about deep learning
## Output
---DEBUG--- EOS generated at 58 position
Deep learning is a powerful tool
That can learn from data and rules
It can recognize patterns and features
And make predictions with accuracy
Deep learning is a fascinating field
That can solve many problems and challenges
It can inspire creativity and curiosity
And make the world a better place

